# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import bokeh as dt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,36.18,69,89,2.75,IS,1715969981
1,1,klyuchi,52.2667,79.1667,57.42,43,14,7.81,RU,1715969981
2,2,pitimbu,-7.4706,-34.8086,75.79,80,46,7.72,BR,1715969981
3,3,palmer,42.1584,-72.3287,75.52,49,75,4.61,US,1715969981
4,4,saldanha,-33.0117,17.9442,60.85,89,95,12.71,ZA,1715969981


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size = 'Humidity',
    scale = 1,
    color = 'City',
    alpha = 0.5,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500
)

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
hotel_df = city_data_df[
    (city_data_df['Max Temp']>70) &
    (city_data_df['Max Temp']<100) &
    (city_data_df['Humidity']<20) 
].copy()

hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,geidam,12.8970,11.9304,95.29,7,0,11.34,NG,1715969982
35,35,faya-largeau,17.9257,19.1043,91.20,10,0,12.57,TD,1715969983
50,50,mansa,29.9833,75.3833,96.91,11,2,7.43,IN,1715969984
53,53,adre,13.4667,22.2000,86.27,11,0,10.33,TD,1715969838
94,94,zouerate,22.7187,-12.4521,88.88,14,0,9.24,MR,1715969986
147,147,at taj,24.2049,23.2857,86.31,15,1,11.45,LY,1715969988
159,159,gereida,11.2811,25.1414,89.74,10,5,8.68,SD,1715969989
168,168,al bawiti,28.3492,28.8659,88.72,17,0,10.89,EG,1715969989
195,195,kot ghulam muhammad,32.3330,74.5480,89.55,17,0,5.66,PK,1715969990
280,280,medina,24.4686,39.6142,97.23,9,20,6.91,SA,1715969847


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:

    # Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,15,geidam,12.8970,11.9304,95.29,7,0,11.34,NG,1715969982,
35,35,faya-largeau,17.9257,19.1043,91.20,10,0,12.57,TD,1715969983,
50,50,mansa,29.9833,75.3833,96.91,11,2,7.43,IN,1715969984,
53,53,adre,13.4667,22.2000,86.27,11,0,10.33,TD,1715969838,
94,94,zouerate,22.7187,-12.4521,88.88,14,0,9.24,MR,1715969986,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 1000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'filter=circle:{lng},{lat},{radius}'
    params["bias"] = f'bias=proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
geidam - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
mansa - nearest hotel: No hotel found
adre - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
gereida - nearest hotel: No hotel found
al bawiti - nearest hotel: No hotel found
kot ghulam muhammad - nearest hotel: No hotel found
medina - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
bechar - nearest hotel: No hotel found
bela - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,15,geidam,12.8970,11.9304,95.29,7,0,11.34,NG,1715969982,No hotel found
35,35,faya-largeau,17.9257,19.1043,91.20,10,0,12.57,TD,1715969983,No hotel found
50,50,mansa,29.9833,75.3833,96.91,11,2,7.43,IN,1715969984,No hotel found
53,53,adre,13.4667,22.2000,86.27,11,0,10.33,TD,1715969838,No hotel found
94,94,zouerate,22.7187,-12.4521,88.88,14,0,9.24,MR,1715969986,No hotel found
147,147,at taj,24.2049,23.2857,86.31,15,1,11.45,LY,1715969988,No hotel found
159,159,gereida,11.2811,25.1414,89.74,10,5,8.68,SD,1715969989,No hotel found
168,168,al bawiti,28.3492,28.8659,88.72,17,0,10.89,EG,1715969989,No hotel found
195,195,kot ghulam muhammad,32.3330,74.5480,89.55,17,0,5.66,PK,1715969990,No hotel found
280,280,medina,24.4686,39.6142,97.23,9,20,6.91,SA,1715969847,No hotel found


In [16]:
geoapify_key

'4f2fc9ae83d4c8dcd365a83eced55d3b'

In [14]:
name_address

{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}

In [60]:
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        data = response.json()
        
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_name = data["features"][0]["properties"]["name"]
            hotel_df.loc[index, "Hotel name"] = hotel_name
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel name"] = "No hotel found"
    else:
        print(f"Failed to retrieve hotels for {row['City']}. Status code: {response.status_code}")

    # Log the search results

Failed to retrieve hotels for georgetown. Status code: 401
Failed to retrieve hotels for longyearbyen. Status code: 401
Failed to retrieve hotels for ola. Status code: 401
Failed to retrieve hotels for puerto ayora. Status code: 401
Failed to retrieve hotels for ouadda. Status code: 401
Failed to retrieve hotels for uturoa. Status code: 401
Failed to retrieve hotels for dudinka. Status code: 401
Failed to retrieve hotels for fort st. john. Status code: 401
Failed to retrieve hotels for anadyr. Status code: 401
Failed to retrieve hotels for ippy. Status code: 401
Failed to retrieve hotels for wailua homesteads. Status code: 401
Failed to retrieve hotels for caleta de carquin. Status code: 401
Failed to retrieve hotels for yellowknife. Status code: 401
Failed to retrieve hotels for port-aux-francais. Status code: 401
Failed to retrieve hotels for kulunda. Status code: 401
Failed to retrieve hotels for adamstown. Status code: 401
Failed to retrieve hotels for bethel. Status code: 401
Fail

In [22]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
%%capture --no-display

#Create map, display size of each point as the percentage of humidity in each city  
#Configure map, display point for every city in the Dataframe
city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size = 'Humidity',
    scale = 1,
    color = 'City',
    alpha = 0.5,
    titles = 'OSM',
    frame_width = 700,
    frame_height = 500
)


%%capture --no-display

#Create map, display size of each point as the percentage of humidity in each city  
#Configure map, display point for every city in the Dataframe
city_data_df.hvplot.points()
city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size = 'Humidity',
    scale = 1,
    color = 'City',
    alpha = 0.5,
    titles = 'OSM',
    frame_width = 700,
    frame_height = 500
    

)
display %map 